In [28]:
# Install required libraries (run this first!)
!pip install faker plotly -q



In [29]:
print("Libraries installed successfully!")
import pandas as pd                        #for working with data tables
import numpy as np                         #for working mith mathematical operations and random numbers
from faker import Faker                    #for creating fake realistic data
from datetime import datetime, timedelta   # for working with dates
import plotly.express as px                #for creating interactive charts
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')         #hide warning messages

print("All libraries imported")
print("today's date: ", datetime.now().date())

Libraries installed successfully!
All libraries imported
today's date:  2026-01-28


In [30]:
#set random seed so we get same data every time
np.random.seed(42)
fake= Faker()
Faker.seed(42)




In [31]:
from faker import Faker # Ensure Faker is imported for standalone execution

def generate_customers(n=1000):
  """
  create fake customer data
  each customer has: ID, Signup date, age, income, credir score, etc
  """
  print(f"creating {n} fake customers...")

  customers = []

  # Ensure fake is initialized here for standalone execution
  fake = Faker()
  Faker.seed(42)

  for i in range(n):
    #create a signup date in the last 3 years
    signup_date = fake.date_between(start_date='-3y', end_date='today')

    customer = {
        "customer_id": f'CUST_{i:05d}',
        "signup_date": signup_date,
        "age": np.random.randint(18,75),
        "income": np.random.uniform(25000, 150000),
        "credit_score": np.random.randint(300,850),
        "account_type": np.random.choice(["basic", "premium", "business"], p=[0.6, 0.3, 0.1]),
        "region": np.random.choice(["North", "South", "East", "West"])
    }
    customers.append(customer)
  df= pd.DataFrame(customers)
  print(f"created {len(df)} customers!")
  return df

#generate the data
customers_df = generate_customers(1000)

# show first 5 rows
print("\n first 5 customers")
customers_df.head()

creating 1000 fake customers...
created 1000 customers!

 first 5 customers


,customer_id,signup_date,age,income,credit_score,account_type,region
0,CUST_00000,2024-12-28,56,124567.873358,570,premium,North
1,CUST_00001,2023-02-24,38,44502.330055,766,basic,East
2,CUST_00002,2023-11-25,41,66713.576392,430,basic,South
3,CUST_00003,2023-09-29,41,129055.330100,685,basic,North
4,CUST_00004,2025-04-14,50,63030.280370,321,basic,North


In [32]:
def generate_transactions(customers_df, avg_transactions= 50):
  """create fake transaction data
  each transaction has: ID, customer, date, amount, type, category
  """
  print(f"Creating transactions (avg {avg_transactions} per customer)...")

  transactions = []

  for idx, customer in customers_df.iterrows():
    #each customer gets random number of transactions (around 50)
    n_transactions = np.random.poisson(avg_transactions)

    signup_date= customer['signup_date']
    days_since_signup = (datetime.now().date() - signup_date).days

    for _ in range(n_transactions):
      #transaction happens sometime after signup
      transaction_date = signup_date + timedelta(days =np.random.randint(0, days_since_signup))

      #transaction amount depends on account type
      #business account spend more, basic account spend less
      if customer["account_type"]== 'business':
        amount = np.random.lognormal(6,1.5)           #higher amount
      elif customer["account_type"]== 'premium':
        amount = np.random.lognormal(5,1.2)           #med amount
      else:
        amount = np.random.lognormal(4,1.0)           #lower amount

      transaction = {
          "transaction_id": fake.uuid4(),
          "customer_id": customer["customer_id"],
          "transaction_date": transaction_date,
          "amount": round(amount,2),
          "transaction_type" : np.random.choice(["purchase", "transfer", "withdrawl", "deposit"], p=[0.5,0.2,0.15,0.15]),
          "merchant_category": np.random.choice(["retail", "dining", "travel", "entertainment", "bills"])
      }
      transactions.append(transaction)
  df= pd.DataFrame(transactions)
  print(f"created {len(df):,} transactions!")
  return df

#generate transactions
transactions_df = generate_transactions(customers_df)

#show first 5 transactions
print("\n first 5 transactions")
transactions_df.head()

Creating transactions (avg 50 per customer)...
created 49,990 transactions!

 first 5 transactions


,transaction_id,customer_id,transaction_date,amount,transaction_type,merchant_category
0,5e09a9ee-af88-4dec-bb1e-143b196f4dfa,CUST_00000,2025-02-03,370.02,withdrawl,retail
1,a5135ea0-fa53-434d-a6d5-901d8b621d41,CUST_00000,2025-10-11,37.52,withdrawl,dining
2,46a02a9b-65ec-4acd-8f80-35f55bd20c98,CUST_00000,2025-01-01,264.58,purchase,dining
3,f2f25eef-1f45-4bfd-b7dc-67e030974b2b,CUST_00000,2025-11-16,64.39,purchase,dining
4,1777e8cb-7468-4b98-92fe-2fded918b3e5,CUST_00000,2025-03-14,306.82,transfer,bills


In [33]:
def generate_loans(customers_df, loan_probability = 0.3):
  """create fake loan data
  30% of customers have taken a loan default risk based on credit score
  """
  print(f" creating loan data...")
  loans =[]

  for idx, customer in customers_df.iterrows():
    #30% chance this customer has a loan
    if np.random.random() < loan_probability:

      #lower credit score = higher chance of default
      #formula: if credit score is 300, default prob= 30%
      # if credit score is 750, default prob = 0%
      default_prob = max(0, (750- customer["credit_score"])/450*0.3)
      is_default= np.random.random() < default_prob

      loan_amount= np.random.uniform(5000,50000) #loan between 5k-50k
      interest_rate= 0.05 + (750 - customer["credit_score"])/450*0.15 #5%-20%

      loan = {

            "loan_id": fake.uuid4(),
            "customer_id": customer["customer_id"],
            "loan_amount": round(loan_amount,2),
            "interest_rate": round(interest_rate,4),
            "loan_date": fake.date_between(start_date= customer["signup_date"], end_date= "today"),
            "loan_term_months": np.random.choice([12,24,36,60]),
            "is_defaulted": is_default ,
            "loan_purpose": np.random.choice(["personal", "business", "education", "home"])

      }

      loans.append(loan)
  df= pd.DataFrame(loans)
  print(f" created {len(df)} loans!")
  return df

#generate loans
loans_df= generate_loans(customers_df)

#show first 5 loans
print("\n First 5 loans:")
loans_df.head()

 creating loan data...
 created 304 loans!

 First 5 loans:


,loan_id,customer_id,loan_amount,interest_rate,loan_date,loan_term_months,is_defaulted,loan_purpose
0,a959f6ca-4f60-4262-b41f-3b6d1aa1783d,CUST_00000,34536.25,0.1100,2025-08-04,12,False,business
1,70c8a500-e73e-4ba2-848f-3b0b351de953,CUST_00001,26980.27,0.0447,2024-09-07,36,False,education
2,fce35a91-99f7-4b74-b037-b51edfb1fe85,CUST_00007,37809.18,0.1933,2024-10-24,36,True,home
3,f7135628-76eb-45d6-a535-7ab50be77773,CUST_00008,20500.84,0.1957,2025-11-17,60,False,home
4,c98502c5-222c-4f6d-929b-234fe8bd37bf,CUST_00010,46195.06,0.1887,2025-09-06,60,True,education


In [34]:
# Let's see what we have
print("="* 60)
print("Data Summary")
print("=" * 60)
print(f"Total Customer: {len(customers_df):,}")
print(f"Total Transactions: {len(transactions_df):,}")
print(f"Total Loans: {len(loans_df):,}")
print(f"Total Transaction Volume ${transactions_df["amount"].sum():,.2f}")
print(f"Average Transaction: ${transactions_df["amount"].mean():,.2f}")
print("="* 60)

#Account type distribution
print("\n Account type distribution:")
print(customers_df["account_type"].value_counts())

#Transaction types
print("\n Transaction Type Distribution:")
print(transactions_df["transaction_type"].value_counts())


Data Summary
Total Customer: 1,000
Total Transactions: 49,990
Total Loans: 304
Total Transaction Volume $13,501,677.78
Average Transaction: $270.09

 Account type distribution:
account_type
basic       617
premium     285
business     98
Name: count, dtype: int64

 Transaction Type Distribution:
transaction_type
purchase     25052
transfer      9968
deposit       7504
withdrawl     7466
Name: count, dtype: int64


In [35]:
generate_loans

<function __main__.generate_loans(customers_df, loan_probability=0.3)>

In [36]:
def calculate_rfm(transactions_df):
  """
  Calculate RFM scores
  R- Recency( how recently did the customer purchase)
  F- Frequency (how often did the customer purchase)
  M- Monetary Value (how much did the customer spend)
  """
  print("Calculating RFM scores...")

  #use the most recent date in our data as reference
  reference_date = transactions_df['transaction_date'].max()

  #group by customer and calculate matrics
  rfm= transactions_df.groupby("customer_id").agg({
      "transaction_date": lambda x: (reference_date - x.max()).days,  #days since last purchased
      "transaction_id": "count",    # no. of transactions
      "amount": "sum"               #total amount spent
  }).rename(columns={
      "transaction_date": "recency",
      "transaction_id": "frequency",
      "amount": "monetary"
  })

  #create score from 1-5(5 is best)
  #for recency: lower is better (bought recently), so we reverse the lable
  rfm["r_score"] = pd.qcut(rfm["recency"],5, labels=[5,4,3,2,1],duplicates= "drop")

  #for frequency: higher is better
  rfm["f_score"] = pd.qcut(rfm["frequency"].rank(method= "first"),5, labels=[1,2,3,4,5], duplicates= "drop")

  #for monerary: higher is better
  rfm["m_score"] = pd.qcut(rfm["monetary"], 5, labels=[1,2,3,4,5], duplicates= "drop")

  #convert to numeric
  rfm["r_score"] = rfm["r_score"].astype(int)
  rfm["f_score"] = rfm["f_score"].astype(int)
  rfm["m_score"] = rfm["m_score"].astype(int)

  # create combined RFM score (eg. "555" for best customer)
  rfm["rfm_score"]= rfm["r_score"].astype(str) + rfm[ "f_score"].astype(str) +rfm["m_score"].astype(str)

  print(f"RFM calculated for {len(rfm)} customers!")
  return rfm

#calculate RFM
rfm_df = calculate_rfm(transactions_df)

#Show examples
print("\n Sample RFM Scores:")
print(rfm_df.head())

print("\n Best customers (RFM= 555):")
print(rfm_df[rfm_df["rfm_score"]== "555"].head())

Calculating RFM scores...
RFM calculated for 1000 customers!

 Sample RFM Scores:
             recency  frequency  monetary  r_score  f_score  m_score rfm_score
customer_id                                                                   
CUST_00000        10         46  12719.80        2        2        4       224
CUST_00001         6         54   3755.35        3        4        1       341
CUST_00002         1         54   4884.63        5        4        3       543
CUST_00003         4         62   6922.82        3        5        4       354
CUST_00004        10         57   3934.20        2        5        2       252

 Best customers (RFM= 555):
             recency  frequency  monetary  r_score  f_score  m_score rfm_score
customer_id                                                                   
CUST_00307         0         57  21383.75        5        5        5       555
CUST_00354         0         57  17171.90        5        5        5       555
CUST_00435         0

In [37]:
def segment_customers(rfm_df):
  """
  Assign customers to segments based on RFM scores
  """
  print("Segmenting Customers")

  def assign_segment(row):
    r,f,m = row["r_score"], row["f_score"], row["m_score"]
    # champions: best customers (high on all metrics)
    if r>=4 and f>=4 and m>=4:
      return "Champion"

    # Loyal customers: good customers
    elif r>=3 and f>=3 and m>=3:
      return "Loyal Customer"

    #New Customers: Recently joined, not yet frequent
    elif r>=4 and f<=2 :
      return "New Customer"

    #At Risk: were good customers, but haven't bought recently
    elif r<=2 and f>=3:
      return "At Risk"

    #Lost: Haven't bought in a long time
    elif r<=2 and f<=2:
      return "Lost"

    #Everyone else
    else:
      return "Potential Loyalists"

  rfm_df["segment"] = rfm_df.apply(assign_segment, axis =1)

  print("Segmentation complete!")
  return rfm_df

# segment customers
rfm_df = segment_customers(rfm_df)

#show segment distribution
print("\n customer segments :")
segment_counts = rfm_df["segment"].value_counts()
print(segment_counts)

print("\n Percentage Distribution:")
print((segment_counts / len(rfm_df)*100).round(2))

Segmenting Customers
Segmentation complete!

 customer segments :
segment
At Risk                216
Loyal Customer         189
Potential Loyalists    180
Lost                   167
New Customer           167
Champion                81
Name: count, dtype: int64

 Percentage Distribution:
segment
At Risk                21.6
Loyal Customer         18.9
Potential Loyalists    18.0
Lost                   16.7
New Customer           16.7
Champion                8.1
Name: count, dtype: float64


In [38]:
def calculate_clv(transactions_df, customers_df):
  """
  calculate customer lifetime value
  CLV= how much money will this customer bring us in the next 12 months?

  Formula: CLV= Avg purchase * Purchase Frequency *time period
  """

  print("calculating Customer Lifetime Value")

  #Avg amount per transaction
  avg_purchase = transactions_df.groupby("customer_id")["amount"].mean()

  #Calculate how long each customer has been active (in months)
  customer_age_months= transactions_df.groupby("customer_id").apply(lambda x: max((x["transaction_date"].max()- x["transaction_date"].min()).days/30,1))

  #Purchase frequency (transaction per month)
  purchase_frequency= transactions_df.groupby("customer_id").size() / customer_age_months

  #CLV = Average Purchase * frequency * 12 months
  clv = avg_purchase * purchase_frequency * 12 # Changed 'months' to '12'

  print(f"CLV calculate for {len(clv)} customers!")
  return clv.fillna(0)

#Calculate CLV
clv_series = calculate_clv(transactions_df, customers_df) # Added customers_df

#Add to RFM dataframe
rfm_df["clv"]= clv_series

#show Top 10 most valuable customers
print("\n Top 10 Most Valuable Customers:")
print(rfm_df.nlargest(10, "clv")[["recency", "frequency", "monetary", "segment","clv"]])

print(f"\n Average CLV: ${rfm_df["clv"].mean():,.2f}")
print(f" Median CLV: ${rfm_df["clv"].median():,.2f}")
print(f" Total Projected Revenue (12 months): ${rfm_df["clv"].sum():,.2f}")

calculating Customer Lifetime Value
CLV calculate for 1000 customers!

 Top 10 Most Valuable Customers:
             recency  frequency   monetary       segment           clv
customer_id                                                           
CUST_00255         0         42  106694.05  New Customer  1.280329e+06
CUST_00321         0         45   65648.86  New Customer  7.877863e+05
CUST_00754         0         52   54802.21      Champion  6.576265e+05
CUST_00200         2         53   49970.90      Champion  5.803072e+05
CUST_00150         0         53   44948.49      Champion  5.393819e+05
CUST_00182         0         40   40995.30  New Customer  4.919436e+05
CUST_00060         0         54   40428.55      Champion  4.851426e+05
CUST_00170         2         57   72879.42      Champion  3.915909e+05
CUST_00900         0         57   31123.35      Champion  3.734802e+05
CUST_00599         0         57   51846.05      Champion  3.521618e+05

 Average CLV: $23,247.52
 Median CLV: $5,92

In [39]:
def calculate_churn_probability(transactions_df, days_threshold= 90):
  """
  calculate churn probability
  if a customer hasn't transacted in 90+ days, they might leave(churn)
  """

  print("Calculating Churn Probability...")

  # Ensure 'transaction_date' is in datetime format for proper .dt accessor functionality
  transactions_df['transaction_date'] = pd.to_datetime(transactions_df['transaction_date'])

  #get the latest date in our data
  latest_date = transactions_df["transaction_date"].max()

  #Find each customer's last transaction date
  last_transaction= transactions_df.groupby("customer_id")["transaction_date"].max()

  #calculate since last transaction
  # Now (latest_date - last_transaction) will be a Series of Timedelta objects (dtype timedelta64[ns])
  # for which the .dt accessor is valid.
  days_since_last = (latest_date - last_transaction).dt.days

  #churn probability: 0% if active, increases with inactivity, caps at 100%
  churn_prob= np.minimum(days_since_last / days_threshold, 1.0) * 100

  print(f" Churn probability calculated!")
  return churn_prob, days_since_last

#calculate churn
churn_prob, days_inactive = calculate_churn_probability(transactions_df)

# ADD to RFM dataframe
rfm_df["churn_probability"] = churn_prob
rfm_df["days_inactive"] = days_inactive

#Show customers at high risk of churning
print("\n High Risk Churn Customers(>80% probability)")
high_risk= rfm_df[rfm_df["churn_probability"]>80].sort_values("churn_probability", ascending = False)
print(high_risk[["recency", "frequency", "monetary", "segment", "churn_probability", "clv"]].head(10))

print(f"\n Churn Statistics")
print(f"Average Churn Probability: {rfm_df["churn_probability"].mean():.2f}%")
print(f"Customers at High Risk (>80%): {len(high_risk)}")

Calculating Churn Probability...
 Churn probability calculated!

 High Risk Churn Customers(>80% probability)
             recency  frequency  monetary  segment  churn_probability  \
customer_id                                                             
CUST_00139        99         48  96525.86     Lost         100.000000   
CUST_00598        91         44   3840.71     Lost         100.000000   
CUST_00290        88         56  70955.49  At Risk          97.777778   

                      clv  
customer_id                
CUST_00139   41417.532300  
CUST_00598    1393.806048  
CUST_00290   25442.207570  

 Churn Statistics
Average Churn Probability: 10.63%
Customers at High Risk (>80%): 3


In [40]:
def cohort_analysis(transactions_df):
  """
  Cohort Analysis: Track retention by signup month
  Show if customers who joined in jan 2024 are still active in Feb, March, etc.
  """

  print("Performing cohort analysis...")

  #convert to datetime and extract month
  transactions_df["transaction_month"] = pd.to_datetime(transactions_df["transaction_date"]).dt.to_period("M")

  #Find 1st purchase month for each customer (their "cohort")
  cohorts = transactions_df.groupby("customer_id")["transaction_month"].min().to_frame("cohort_month")

  #Merge cohort info back to transactions
  transactions_with_cohort = transactions_df.merge(cohorts, left_on= "customer_id", right_index = True)

  #calculate which period number this is for the customer (0 = first month, 1= second month etc.)
  transactions_with_cohort["period_number"] = (transactions_with_cohort["transaction_month"]- transactions_with_cohort["cohort_month"]).apply(lambda x :x.n)

  #count unique active customers per cohort per period
  # Fixed typo: changed 'period_numer' to 'period_number'
  cohort_data = transactions_with_cohort.groupby(["cohort_month", "period_number"])["customer_id"].nunique()
  cohort_data = cohort_data.reset_index()
  cohort_pivot = cohort_data.pivot(index= "cohort_month", columns= "period_number", values= "customer_id")

  #Calculate retention rate (% of original cohort still active)
  cohort_size= cohort_pivot.iloc[:, 0] #Number of customers in month 0
  retention = cohort_pivot.divide(cohort_size, axis=0)*100

  print(f"Cohort Analysis Complete!")
  return retention

#performance cohort analysis
retention_df = cohort_analysis(transactions_df)

#Show retention matrix
print("\n Retention Rate Matrix (%):")
print("Rows= Cohort Month | Columns= Month Since Signup")
print(retention_df.round(1))

Performing cohort analysis...
Cohort Analysis Complete!

 Retention Rate Matrix (%):
Rows= Cohort Month | Columns= Month Since Signup
period_number     0      1      2      3      4      5      6      7      8   \
cohort_month                                                                   
2023-01        100.0   50.0   50.0  100.0  100.0  100.0  100.0   50.0   50.0   
2023-02        100.0   66.7   86.7   93.3   80.0   66.7   73.3   66.7   60.0   
2023-03        100.0   83.3   94.4   77.8   77.8   72.2   66.7   77.8   77.8   
2023-04        100.0   65.2   65.2   87.0   73.9   78.3   82.6   78.3   73.9   
2023-05        100.0   85.7   71.4   78.6   82.1   82.1   71.4   71.4   75.0   
2023-06        100.0   80.0   76.7   80.0   80.0   73.3   80.0   76.7   73.3   
2023-07        100.0   81.8   77.3   72.7   86.4   81.8   86.4   72.7   77.3   
2023-08        100.0   81.0  100.0  100.0   85.7   90.5   76.2   76.2   95.2   
2023-09        100.0   85.7   78.6   82.1   85.7   75.0  100.0   8

In [41]:
# Pie chart of customer segments
fig= px.pie(
    values= rfm_df["segment"].value_counts().values,
    names= rfm_df["segment"].value_counts().index,
    title= "Customer Segmentation Distribution",
    color_discrete_sequence=px.colors.qualitative.Set3
)

fig.update_traces(textposition= "inside", textinfo = "percent + label")
fig.update_layout(height= 500, width= 700)
fig.show()

print("What Each Segment Means:")
print("Champions: Our VIPs- market Premium Product to them")
print("At Risk: Need win-back campaigns urgently")
print("New Customers: Focus on onboarding and engagement")
print("Lost: Consider if reactivation is worth the cost")

What Each Segment Means:
Champions: Our VIPs- market Premium Product to them
At Risk: Need win-back campaigns urgently
New Customers: Focus on onboarding and engagement
Lost: Consider if reactivation is worth the cost


In [42]:
segment_clv = rfm_df.groupby("segment").agg({
    "clv" : "mean",
    "monetary": "mean",
    "frequency": "mean",
    "recency": "mean"
}).round(2)

fig = px.bar(segment_clv,
             y= "clv",
             title= "Average Customer Lifetime Value by Segment",
             color= "clv",
             color_continuous_scale= "Blues"
             )

fig.update_layout(height=500, width= 800, showlegend=False)
fig.show()

print("\n Segment Metrics:")
print(segment_clv)


 Segment Metrics:
                          clv  monetary  frequency  recency
segment                                                    
At Risk              10060.22  16140.17      54.42    20.10
Champion             96946.28  29383.14      56.96     1.05
Lost                  6458.81  10817.13      42.57    22.69
Loyal Customer       21816.13  14573.73      55.14     2.74
New Customer         38661.31  11304.07      43.10     1.06
Potential Loyalists   8686.45   6592.72      49.41     3.72


In [43]:
 #Scatter plot: CLV vs Churn Probability
fig = px.scatter(
     rfm_df,
     x= "churn_probability",
     y= "clv",
     color= "segment",
     size= "monetary",
     hover_data = ["recency", "frequency"],
     title= "Customer Value vs Churn Risk",
     labels= {"churn_probability": "Churn Probability(%)", "clv": "Customer Lifetime Value ($)"}
     )

fig.update_layout(height= 600, width= 900)
fig.show()

print("Top right quadrant:")
print("High CLV + High Churn Risk = URGENT ACTION NEEDED!")
print("These are Valuable customers about to leave.")

Top right quadrant:
High CLV + High Churn Risk = URGENT ACTION NEEDED!
These are Valuable customers about to leave.


In [44]:
# Create heatmap of retention rates (IMPROVED VERSION)

# Only show cohorts and periods that have meaningful data
# Remove cohorts that are too new (less than 2 periods of data)
retention_df_clean = retention_df.copy()

# Count how many non-NaN values each cohort has
cohort_data_count = retention_df_clean.notna().sum(axis=1)

# Keep only cohorts with at least 3 data points (Month 0, 1, and 2)
retention_df_clean = retention_df_clean[cohort_data_count >= 3]

# If we have too few cohorts after filtering, use all cohorts
if len(retention_df_clean) < 3:
    retention_df_clean = retention_df.copy()
    print(" Note: Showing all cohorts (some are very recent)")

# Limit to first 12 months (or however many columns we have)
max_periods = min(12, len(retention_df_clean.columns))
retention_df_clean = retention_df_clean.iloc[:, :max_periods]

# Replace NaN with empty string for display (won't show in heatmap)
retention_display = retention_df_clean.fillna('')

# Create custom text for heatmap (show percentage or empty)
text_values = []
for i in range(len(retention_display)):
    row_text = []
    for j in range(len(retention_display.columns)):
        val = retention_display.iloc[i, j]
        if val == '':
            row_text.append('')  # Empty cell
        else:
            row_text.append(f'{val:.1f}%')  # Show percentage
    text_values.append(row_text)

# Create the heatmap
fig = go.Figure(data=go.Heatmap(
    z=retention_df_clean.values,  # Use original values (NaN will be colored differently)
    x=[f'Month {i}' for i in retention_df_clean.columns],
    y=[str(x) for x in retention_df_clean.index],
    text=text_values,
    texttemplate='%{text}',
    textfont={"size": 11},
    colorscale=[
        [0, '#d73027'],      # Red (0% - poor retention)
        [0.3, '#fc8d59'],    # Orange (30%)
        [0.5, '#fee090'],    # Yellow (50%)
        [0.7, '#e0f3f8'],    # Light blue (70%)
        [1, '#4575b4']       # Dark blue (100% - excellent retention)
    ],
    colorbar=dict(
        title="Retention %",
        ticksuffix="%"
    ),
    zmid=50,  # Center the color scale at 50%
    zmin=0,
    zmax=100
))

fig.update_layout(
    title={
        'text': 'Cohort Retention Analysis (%)<br><sub>Green = Good Retention | Red = Poor Retention</sub>',
        'x': 0.5,
        'xanchor': 'center'
    },
    xaxis_title='Months Since First Purchase',
    yaxis_title='Cohort (Signup Month)',
    height=600,
    width=1000,
    xaxis={'side': 'bottom'},
    yaxis={'autorange': 'reversed'}  # Most recent cohorts at top
)

fig.show()

# Print insights based on the heatmap
print("\n" + "=" * 70)
print(" COHORT RETENTION INSIGHTS")
print("=" * 70)

# Insight 1: Overall retention trend
if 1 in retention_df.columns:
    month_1_retention = retention_df[1].dropna()
    if len(month_1_retention) > 0:
        avg_month_1 = month_1_retention.mean()
        print(f"\n1 MONTH-1 RETENTION (First Month After Signup)")
        print(f"   • Average retention: {avg_month_1:.1f}%")

        if avg_month_1 < 60:
            print(f"   • Status:  CRITICAL - Very low retention")
            print(f"    Action: Immediate focus on onboarding improvements")
        elif avg_month_1 < 75:
            print(f"   • Status:  CONCERNING - Below industry standard")
            print(f"    Action: Review first-month user experience")
        elif avg_month_1 < 85:
            print(f"   • Status:  GOOD - Solid retention")
            print(f"    Action: Continue current onboarding strategies")
        else:
            print(f"   • Status:  EXCELLENT - Outstanding retention")
            print(f"    Action: Document and replicate success factors")

# Insight 2: Month-3 retention (if available)
if 3 in retention_df.columns:
    month_3_retention = retention_df[3].dropna()
    if len(month_3_retention) > 0:
        avg_month_3 = month_3_retention.mean()
        print(f"\n2 MONTH-3 RETENTION (Long-term Engagement)")
        print(f"   • Average retention: {avg_month_3:.1f}%")

        if avg_month_1 > 0 and avg_month_3 > 0:
            drop_off = avg_month_1 - avg_month_3
            drop_off_pct = (drop_off / avg_month_1) * 100
            print(f"   • Drop-off from Month 1 to Month 3: {drop_off:.1f}% ({drop_off_pct:.1f}%)")

            if drop_off_pct > 40:
                print(f"    Action: HIGH churn in Months 2-3, investigate user pain points")
            elif drop_off_pct > 25:
                print(f"    Action: Moderate churn, enhance engagement features")
            else:
                print(f"    Action: Stable retention, maintain current strategies")

# Insight 3: Cohort comparison (are newer cohorts better?)
if len(retention_df) >= 3 and 2 in retention_df.columns:
    # Compare oldest 3 cohorts vs newest 3 cohorts (that have Month 2 data)
    cohorts_with_month_2 = retention_df[2].dropna()

    if len(cohorts_with_month_2) >= 6:
        oldest_3 = cohorts_with_month_2.head(3).mean()
        newest_3 = cohorts_with_month_2.tail(3).mean()

        print(f"\n3 COHORT PERFORMANCE TREND")
        print(f"   • Oldest cohorts (Month-2 retention): {oldest_3:.1f}%")
        print(f"   • Newest cohorts (Month-2 retention): {newest_3:.1f}%")
        print(f"   • Change: {newest_3 - oldest_3:+.1f} percentage points")

        if newest_3 > oldest_3 + 5:
            print(f"   • Status:  IMPROVING - Product changes are working!")
            print(f"    Action: Document successful improvements, continue optimization")
        elif newest_3 < oldest_3 - 5:
            print(f"   • Status:  DECLINING - Product quality decreasing")
            print(f"    Action: Review recent changes, identify negative factors")
        else:
            print(f"   • Status:  STABLE - Consistent performance")
            print(f"    Action: Test new retention initiatives to improve")
    elif len(cohorts_with_month_2) >= 3:
        avg_retention = cohorts_with_month_2.mean()
        print(f"\n3 COHORT PERFORMANCE")
        print(f"   • Average Month-2 retention: {avg_retention:.1f}%")
        print(f"   • Note: Limited cohort history, continue monitoring")

# Insight 4: Identify best and worst performing cohorts
if len(retention_df) >= 2 and 2 in retention_df.columns:
    month_2_data = retention_df[2].dropna()

    if len(month_2_data) >= 2:
        best_cohort = month_2_data.idxmax()
        best_retention = month_2_data.max()
        worst_cohort = month_2_data.idxmin()
        worst_retention = month_2_data.min()

        print(f"\n4 COHORT COMPARISON")
        print(f"   • Best performing cohort: {best_cohort} ({best_retention:.1f}% at Month 2)")
        print(f"   • Worst performing cohort: {worst_cohort} ({worst_retention:.1f}% at Month 2)")
        print(f"   • Performance gap: {best_retention - worst_retention:.1f} percentage points")

        if best_retention - worst_retention > 20:
            print(f"    Action: Investigate what made {best_cohort} successful")
            print(f"            Study external factors (seasonality, marketing campaigns)")

# Insight 5: Retention benchmark
print(f"\n5 INDUSTRY CONTEXT")
print(f"   • Fintech/SaaS benchmarks:")
print(f"     - Excellent: >80% Month-1, >60% Month-3")
print(f"     - Good: 70-80% Month-1, 50-60% Month-3")
print(f"     - Poor: <70% Month-1, <50% Month-3")

if 1 in retention_df.columns and 3 in retention_df.columns:
    m1 = retention_df[1].dropna().mean()
    m3_data = retention_df[3].dropna()

    if len(m3_data) > 0:
        m3 = m3_data.mean()

        if m1 > 80 and m3 > 60:
            overall = " EXCELLENT"
        elif m1 > 70 and m3 > 50:
            overall = " GOOD"
        else:
            overall = " NEEDS IMPROVEMENT"

        print(f"   • Your performance: {overall}")

print("\n" + "=" * 70)


 COHORT RETENTION INSIGHTS

1 MONTH-1 RETENTION (First Month After Signup)
   • Average retention: 90.1%
   • Status:  EXCELLENT - Outstanding retention
    Action: Document and replicate success factors

2 MONTH-3 RETENTION (Long-term Engagement)
   • Average retention: 92.0%
   • Drop-off from Month 1 to Month 3: -1.9% (-2.1%)
    Action: Stable retention, maintain current strategies

3 COHORT PERFORMANCE TREND
   • Oldest cohorts (Month-2 retention): 77.0%
   • Newest cohorts (Month-2 retention): 100.0%
   • Change: +23.0 percentage points
   • Status:  IMPROVING - Product changes are working!
    Action: Document successful improvements, continue optimization

4 COHORT COMPARISON
   • Best performing cohort: 2023-08 (100.0% at Month 2)
   • Worst performing cohort: 2023-01 (50.0% at Month 2)
   • Performance gap: 50.0 percentage points
    Action: Investigate what made 2023-08 successful
            Study external factors (seasonality, marketing campaigns)

5 INDUSTRY CONTEXT
   •

In [45]:
print("=" * 70)
print("KEY BUSINESS INSIGHTS")
print("=" * 70)

# Insight 1: Customer Segments
champions = len(rfm_df[rfm_df['segment'] == 'Champion'])
at_risk = len(rfm_df[rfm_df['segment'] == 'At Risk'])
lost = len(rfm_df[rfm_df['segment'] == 'Lost'])

print(f"\n 1 CUSTOMER SEGMENTATION")
print(f"   • Champions: {champions} ({champions/len(rfm_df)*100:.1f}%)")
print(f"   • At Risk: {at_risk} ({at_risk/len(rfm_df)*100:.1f}%)")
print(f"   • Lost: {lost} ({lost/len(rfm_df)*100:.1f}%)")
print(f"   Action: Focus retention efforts on {at_risk} 'At Risk' customers")

# Insight 2: Revenue Concentration
top_20_pct_clv = rfm_df.nlargest(int(len(rfm_df)*0.2), 'clv')['clv'].sum()
total_clv = rfm_df['clv'].sum()
concentration = (top_20_pct_clv / total_clv * 100)

print(f"\n 2 REVENUE CONCENTRATION")
print(f"   • Top 20% of customers generate {concentration:.1f}% of revenue")
print(f"   Action: VIP program for high-value customers")

# Insight 3: Churn Risk
high_value_at_risk = rfm_df[(rfm_df['clv'] > rfm_df['clv'].median()) &
                             (rfm_df['churn_probability'] > 70)]
print(f"\n 3 CHURN RISK")
print(f"   • {len(high_value_at_risk)} high-value customers at risk of churning")
print(f"   • Potential revenue loss: ${high_value_at_risk['clv'].sum():,.2f}")
print(f"   Action: Immediate win-back campaign for these customers")

# Insight 4: Cohort Performance

print(f"\n 4 RETENTION INSIGHTS")

# Check if we have any cohorts with Month 3 data
if 3 in retention_df.columns:
    # Get all cohorts that have Month 3 data (not NaN)
    cohorts_with_month_3 = retention_df[3].dropna()

    if len(cohorts_with_month_3) > 0:
        avg_month_3_retention = cohorts_with_month_3.mean()
        print(f"   • Average Month-3 retention: {avg_month_3_retention:.1f}%")

        # Provide business insight based on retention
        if avg_month_3_retention < 50:
            print(f"    Action: URGENT - Improve onboarding (retention below 50%)")
        elif avg_month_3_retention < 70:
            print(f"    Action: Enhance user experience in first 3 months")
        else:
            print(f"    Action: Good retention! Focus on maintaining quality")
    else:
        print(f"   • Month-3 retention: Not enough data yet")
        print(f"    Note: No cohorts have reached 3 months of age")
else:
    print(f"   • Month-3 retention: Not available (cohorts too new)")
    print(f"   Note: Wait for older cohorts to analyze Month-3 retention")

# Alternative: Show Month-1 retention instead (more cohorts will have this)
if 1 in retention_df.columns:
    cohorts_with_month_1 = retention_df[1].dropna()
    if len(cohorts_with_month_1) > 0:
        avg_month_1_retention = cohorts_with_month_1.mean()
        print(f"   • Average Month-1 retention: {avg_month_1_retention:.1f}%")

        if avg_month_1_retention < 60:
            print(f"    Action: Critical - Fix immediate user experience")
        elif avg_month_1_retention < 80:
            print(f"    Action: Improve first-month engagement")

# Insight 5: Loan Risk
if len(loans_df) > 0:
    default_rate = loans_df['is_defaulted'].mean() * 100
    print(f"\n 5 LOAN PORTFOLIO")
    print(f"   • Default rate: {default_rate:.2f}%")
    print(f"   • Total loan volume: ${loans_df['loan_amount'].sum():,.2f}")

    # Merge with customers to get credit scores
    loans_with_score = loans_df.merge(customers_df[['customer_id', 'credit_score']], on='customer_id')
    avg_default_score = loans_with_score[loans_with_score['is_defaulted']]['credit_score'].mean()
    print(f"   • Avg credit score of defaulters: {avg_default_score:.0f}")
    print(f"   Action: Tighten lending criteria for scores below {avg_default_score:.0f}")

print("\n" + "=" * 70)

KEY BUSINESS INSIGHTS

 1 CUSTOMER SEGMENTATION
   • Champions: 81 (8.1%)
   • At Risk: 216 (21.6%)
   • Lost: 167 (16.7%)
   Action: Focus retention efforts on 216 'At Risk' customers

 2 REVENUE CONCENTRATION
   • Top 20% of customers generate 79.7% of revenue
   Action: VIP program for high-value customers

 3 CHURN RISK
   • 3 high-value customers at risk of churning
   • Potential revenue loss: $97,049.64
   Action: Immediate win-back campaign for these customers

 4 RETENTION INSIGHTS
   • Average Month-3 retention: 92.0%
    Action: Good retention! Focus on maintaining quality
   • Average Month-1 retention: 90.1%

 5 LOAN PORTFOLIO
   • Default rate: 12.17%
   • Total loan volume: $8,013,032.08
   • Avg credit score of defaulters: 430
   Action: Tighten lending criteria for scores below 430



In [46]:
#Merge everything into one master dataframe for analysis
master_df = customers_df.merge(rfm_df, left_on='customer_id', right_index=True, how='left')

print("Master Dataset Created!")
print(f"Rows: {len(master_df)}")
print(f"Columns: {len(master_df.columns)}")

# Show sample
print("\n Sample of Master Dataset:")
master_df.head()

# To download as CSV (click the folder icon on the left to find it)
# master_df.to_csv('customer_analytics.csv', index=False)
# print("File saved as 'customer_analytics.csv'")

Master Dataset Created!
Rows: 1000
Columns: 18

 Sample of Master Dataset:


,customer_id,signup_date,age,income,credit_score,account_type,region,recency,frequency,monetary,r_score,f_score,m_score,rfm_score,segment,clv,churn_probability,days_inactive
0,CUST_00000,2024-12-28,56,124567.873358,570,premium,North,10,46,12719.80,2,2,4,224,Lost,12018.708661,11.111111,10
1,CUST_00001,2023-02-24,38,44502.330055,766,basic,East,6,54,3755.35,3,4,1,341,Potential Loyalists,1282.662239,6.666667,6
2,CUST_00002,2023-11-25,41,66713.576392,430,basic,South,1,54,4884.63,5,4,3,543,Loyal Customer,2286.692848,1.111111,1
3,CUST_00003,2023-09-29,41,129055.330100,685,basic,North,4,62,6922.82,3,5,4,354,Loyal Customer,2942.402834,4.444444,4
4,CUST_00004,2025-04-14,50,63030.280370,321,basic,North,10,57,3934.20,2,5,2,252,At Risk,5245.600000,11.111111,10


In [47]:
print("=" * 70)

print("""
1 PROJECT OVERVIEW:
   "I built a customer analytics system for fintech companies. It analyzes
   transaction patterns, segments customers, predicts churn, and calculates
   lifetime value. This helps business analysts make data-driven decisions."

2 TECHNICAL APPROACH:
   "I generated synthetic data for 1,000 customers and 50,000 transactions.
   Then I implemented:
   • RFM Analysis to score customers on Recency, Frequency, and Monetary value
   • Customer Segmentation using business rules (Champions, At Risk, etc.)
   • CLV calculation to identify high-value customers
   • Churn prediction based on inactivity patterns
   • Cohort analysis to track retention over time"

3 KEY INSIGHTS:
   "The analysis revealed that 20% of customers drive 80% of revenue,
   we have 216 customers at high risk of churning who represent $97,049 in value,
   and Month-3 retention is 92%, suggesting focus on maintaining quality"

4 BUSINESS IMPACT:
   "These insights enable targeted actions:
   • Win-back campaigns for 'At Risk' high-value customers
   • VIP programs for 'Champions'
   • Improved onboarding to boost Month-3 retention
   • Risk-based lending using default prediction"

5 TOOLS & SKILLS:
    Python (pandas, numpy)
    Data Analysis & Visualization (Plotly)
    Business Metrics (RFM, CLV, Churn, Cohort Analysis)
    Statistical Thinking
    Business Acumen (translating data to action)
""")

print("=" * 70)


1 PROJECT OVERVIEW:
   "I built a customer analytics system for fintech companies. It analyzes
   transaction patterns, segments customers, predicts churn, and calculates
   lifetime value. This helps business analysts make data-driven decisions."

2 TECHNICAL APPROACH:
   "I generated synthetic data for 1,000 customers and 50,000 transactions.
   Then I implemented:
   • RFM Analysis to score customers on Recency, Frequency, and Monetary value
   • Customer Segmentation using business rules (Champions, At Risk, etc.)
   • CLV calculation to identify high-value customers
   • Churn prediction based on inactivity patterns
   • Cohort analysis to track retention over time"

3 KEY INSIGHTS:
   "The analysis revealed that 20% of customers drive 80% of revenue,
   we have 216 customers at high risk of churning who represent $97,049 in value,
   and Month-3 retention is 92%, suggesting focus on maintaining quality"

4 BUSINESS IMPACT:
   "These insights enable targeted actions:
   • Win-bac